In [1]:
! pip install bs4 langchain_community tiktoken langchain-openai langchainhub chromadb langchain

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY', '')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', '')

In [4]:
# imports
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [5]:
# Indexing

# Load documents
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

Splitting the document

In [6]:
# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
splits = text_splitter.split_documents(docs)

VectorStores

In [7]:
# Embed
vectorstore = Chroma.from_documents(
    documents=splits, 
    embedding=OpenAIEmbeddings(),
    persist_directory="./chroma_db"
)

In [8]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [9]:
prompt_hub_rag = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model = "gpt-3.5-turbo" , temperature=0.5)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [10]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_hub_rag
    | llm
    | StrOutputParser()
)

In [11]:
rag_chain.invoke("What is Task Decomposition?")

'Task decomposition is the process of breaking down a complex task into smaller, more manageable sub-tasks. This approach helps in organizing and prioritizing tasks to achieve the overall goal efficiently. Task decomposition is commonly used in project management and software development.'

# Retrieval

In [ ]:
docs = retriever.invoke("What is Task Decomposition?")

In [ ]:
len(docs)

1

# Generation

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context:
{context}


Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

prompt


ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\n\nQuestion: {question}\n'))])

In [ ]:
chain = prompt | llm

In [ ]:
chain.invoke({"context": docs, "question": "What is Task Decomposition?"})

AIMessage(content='Task Decomposition refers to breaking down a complex task into smaller, more manageable sub-tasks or components.', response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 60, 'total_tokens': 81}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-16fd03c7-a712-495b-964c-843e08c1f997-0')

# Query Translation -- Multi-Query

In [ ]:
template = """You are an AI language model assistant. Your task is to generate five different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search.
Provide these alternative questions separated by newlines. Original question: {question}
"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

In [ ]:
generate_queries = (
    prompt_perspectives
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)


In [ ]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    flatten_docs = [dumps(doc) for sublist in documents for doc in sublist]
    unique_docs = list(set(flatten_docs))
    return [loads(doc) for doc in unique_docs]

question = "What is Task Decomposition for LLM agents?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question": question})
len(docs)

/Users/tush/miniconda3/envs/ds/lib/python3.12/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


1

In [ ]:
from operator import itemgetter

template = """Answer the following question based on this context: 

{context}

Question: {question}"""
prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain, "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)
final_rag_chain.invoke({"question": question})

'Task decomposition for LLM agents involves breaking down large tasks into smaller, manageable subgoals.'

# Query Translation -- RAG Fusion

In [12]:
from langchain.prompts import ChatPromptTemplate

template = """You are helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to {question}\n
Output (4 queries):"""

prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_rag_fusion
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [14]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    fused_scores = {}

    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)

            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0

            fused_scores[doc_str] += 1/(rank+k)

    return [
        (loads(doc), score) 
        for doc, score in sorted(fused_scores.items(), key = lambda x: x[1], reverse=True)
    ]
 

In [16]:
question = "What is Task Decomposition for LLM agents?"
retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

3

In [18]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

template = """Answer the following question based on this context:

{context}

Question: {question}"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question": question})

'Task decomposition for LLM agents involves breaking down large tasks into smaller, manageable subgoals using simple prompting such as "Steps for XYZ."'

# Query Translation -- Decomposition

In [19]:
from langchain.prompts import ChatPromptTemplate

template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to {question}\n
Output (3 queries):
"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [21]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model = "gpt-3.5-turbo" , temperature=0.5)

generate_queries_decomposition = (
    prompt_decomposition
    | llm
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

question = "What are the main components of an LLM-powered autonomous agent system?"
questions = generate_queries_decomposition.invoke({"question": question})

In [22]:
questions

['1. What is LLM technology and how does it work in autonomous agent systems?',
 '2. What are the specific components that make up an LLM-powered autonomous agent system?',
 '3. How do the main components of an LLM-powered autonomous agent system interact with each other to enable autonomous functionality?']

In [23]:
template = """Here is the question you need to answer: 

\n -- \n   {question} \n -- \n

Here is any available background question + answer pairs:

\n -- \n   {q_a_pairs} \n -- \n

Here is additional context relevant to the question:

\n -- \n   {context} \n -- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [24]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

def format_qa_pair(question, answer):
    return f"Question: {question}\nAnswer: {answer}\n\n".strip()


In [27]:
llm = ChatOpenAI(model = "gpt-3.5-turbo" , temperature=0.5)

q_a_pairs = ""
for q in questions:

    rag_chain = (
        {"context": itemgetter("question") | retriever, 
         "question": itemgetter("question"),
         "q_a_pairs": itemgetter("q_a_pairs")}
        | decomposition_prompt
        | llm
        | StrOutputParser()
    )

    answer = rag_chain.invoke({"question": q, "q_a_pairs": q_a_pairs})
    q_a_pair = format_qa_pair(q, answer)
    q_a_pairs += "\n--\n" + q_a_pair

In [28]:
answer

"The main components of an LLM-powered autonomous agent system interact with each other in a cohesive manner to enable autonomous functionality. \n\n1. Large Language Models (LLMs) interact with Natural Language Processing (NLP) capabilities to understand and generate human language effectively. This interaction allows the autonomous agent to process and respond to natural language inputs from users.\n\n2. The communication interface facilitates the interaction between the autonomous agent and users through natural language commands and responses. This interface serves as the bridge for communication between the agent and the human user.\n\n3. Decision-making algorithms utilize the input from LLMs and NLP capabilities to make informed choices based on the language inputs received. This interaction enables the autonomous agent to make intelligent decisions autonomously.\n\n4. Task-specific modules work in conjunction with LLMs, NLP capabilities, and decision-making algorithms to perform

# Query Translation -- Step Back